In [1]:
import SimpleITK as sitk
import os
import csv
import numpy as np
from util import XyzTuple, xyz2irc, logging, getCache
import collections
from collections import namedtuple  
import glob
from concurrent.futures import ThreadPoolExecutor
from pylidc_func import masks_build
from make_mask import aug_mask
from segmentation import segment, init_segment_model

2023-12-09 22:27:01,846 INFO     pid:11192 numexpr.utils:148:_init_num_threads Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-12-09 22:27:01,859 INFO     pid:11192 numexpr.utils:160:_init_num_threads NumExpr defaulting to 8 threads.


In [2]:
CandidateInfoTuple = namedtuple('CandidateInfoTuple', 'isNodule_bool, hasAnnotation_bool, isMal_bool, diameter_mm, series_uid, center_xyz')
def getCandidateInfoList(requireOnDisk_bool=True):
    # We construct a set with all series_uids that are present on disk.
    # This will let us use the data, even if we haven't downloaded all of
    # the subsets yet.
    mhd_list = glob.glob('C:/Users/oplab/Desktop/Luna16_data/Luna16_img/subset*/*.mhd')
    presentOnDisk_set = {os.path.split(p)[-1][:-4] for p in mhd_list}

    candidateInfo_list = []
    with open('C:/Users/oplab/Desktop/Luna16_data/annotations_with_malignancy.csv', "r") as f:
        for row in list(csv.reader(f))[1:]:
            series_uid = row[0]
            annotationCenter_xyz = tuple([float(x) for x in row[1:4]])
            annotationDiameter_mm = float(row[4])
            isMal_bool = {'False': False, 'True': True}[row[5]] #it record the malignancy or not

            if series_uid not in presentOnDisk_set and requireOnDisk_bool:
                continue

            candidateInfo_list.append(
                CandidateInfoTuple(
                    True,
                    True,
                    isMal_bool,
                    annotationDiameter_mm,
                    series_uid,
                    annotationCenter_xyz,
                )
            )

    with open('C:/Users/oplab/Desktop/Luna16_data/candidates_V2.csv', "r") as f:
        for row in list(csv.reader(f))[1:]:
            series_uid = row[0]

            if series_uid not in presentOnDisk_set and requireOnDisk_bool:
                continue

            isNodule_bool = bool(int(row[4]))
            candidateCenter_xyz = tuple([float(x) for x in row[1:4]])

            if not isNodule_bool:
                candidateInfo_list.append(
                    CandidateInfoTuple(
                        False,
                        False,
                        False,
                        0.0,
                        series_uid,
                        candidateCenter_xyz,
                    )
                )

    candidateInfo_list.sort(reverse=True)
    return candidateInfo_list

In [3]:
def getCandidateInfoDict(requireOnDisk_bool=True):  #把candidateInfoList包成Dict
    candidateInfo_list = getCandidateInfoList(requireOnDisk_bool)
    candidateInfo_dict = {}

    for candidateInfo_tup in candidateInfo_list:
        candidateInfo_dict.setdefault(candidateInfo_tup.series_uid,
                                      []).append(candidateInfo_tup)

    return candidateInfo_dict

In [4]:
candidate_dict =  getCandidateInfoDict()
len(candidate_dict)

888

In [5]:
# write_root = "D://LUNA//Luna16_AugData"
# if not os.path.exists(write_root):
#     os.mkdir("D://LUNA//Luna16_AugData")
    
# append = True
# # split to train test
# for sub_root in os.listdir("C://Users//oplab//Desktop//Luna16_data//Luna16_img//"):
#     root = os.path.join("E:\LUNA\Luna_Data", sub_root)
#     print(root)
#     for path in os.listdir(root):
#         break
#         # npy_path = glob.glob('./augmented_segmented/{}.npy'.format(series_uid))
#         npy_path = glob.glob(os.path.join(write_root, sub_root, path) + ''.format(series_uid))
#         if (npy_path != [] and append):
#             print("pass")
#             continue
#         mhd_path = glob.glob('C:/Users/oplab/Desktop/Luna16_data/Luna16_img/subset*/{}.mhd'.format(series_uid))
#         ct_mhd = sitk.ReadImage(mhd_path)
#         if ct_mhd.GetDimension()==4 and ct_mhd.GetSize()[3]==1:
#             ct_mhd = ct_mhd[...,0]
#         origin = sitk.GetArrayFromImage(ct_mhd)
#         print(mhd_path)
#         augmented = augment(origin)
#         # segmented = segment(augmented)
#         # hu_a = np.array(segmented, dtype=np.float32)
#         hu_a = np.array(augmented, dtype=np.float32)
#         # np.save("./augmented/{}.npy".format(series_uid), hu_a)
#         # np.save("./augmented_segmented/{}.npy".format(series_uid), hu_a)
#         # Load or create a SimpleITK image
#         current_origin = ct_mhd.GetOrigin()
#         current_spacing = ct_mhd.GetSpacing()

#         sitk_image = sitk.GetImageFromArray(augmented)
#         # image_short = sitk.Cast(image, sitk.sitkInt16)

#         # Set the image origin, spacing, and direction (modify as needed)
#         sitk_image.SetOrigin((current_origin[0], current_origin[1], current_origin[2]))
#         sitk_image.SetSpacing((current_spacing[0], current_spacing[1], current_spacing[2] / 2))

#         # Save the image as a MetaImage file
#         if not os.path.exists(os.path.join(write_root, sub_root)):
#             os.mkdir(os.path.join(write_root, sub_root))
#         sitk.WriteImage(sitk_image, os.path.join(write_root, sub_root, path)) 

In [6]:
# npy_path = glob.glob(os.path.join(write_root, sub_root, path) + ''.format(series_uid))

In [5]:
model_best, device = init_augment_model()
# model, device = init_segment_model()

NameError: name 'init_augment_model' is not defined

In [8]:
# Function to process a single image
def process_image(sub_root, path, write_root_1, write_root_2, append):
    raw_path_1 = os.path.join(write_root_1, sub_root, path)
    exist_path_1 = glob.glob(os.path.join(write_root_1, sub_root, path))
    raw_path_2 = os.path.join(write_root_2, sub_root, path)
    exist_path_2 = glob.glob(os.path.join(write_root_2, sub_root, path))
    
    if exist_path_1 and append:
        print("pass")
        return
    
    imagePath = os.path.join(root, path)
    ct_mhd = sitk.ReadImage(imagePath)
    
    if ct_mhd.GetDimension() == 4 and ct_mhd.GetSize()[3] == 1:
        ct_mhd = ct_mhd[..., 0]
    
    origin = sitk.GetArrayFromImage(ct_mhd)
    augmented = augment(origin)
    hu_a = np.array(augmented, dtype=np.float32)
    
    current_origin = ct_mhd.GetOrigin()
    current_spacing = ct_mhd.GetSpacing()
    
    sitk_image = sitk.GetImageFromArray(augmented)
    sitk_image.SetOrigin((current_origin[0], current_origin[1], current_origin[2]))
    sitk_image.SetSpacing((current_spacing[0], current_spacing[1], current_spacing[2] / 2))
    
    if not os.path.exists(os.path.join(write_root_1, sub_root)):
        os.makedirs(os.path.join(write_root_1, sub_root))
    
    print(os.path.join(write_root_1, sub_root, path))
    sitk.WriteImage(sitk_image, raw_path_1)


In [10]:
from tqdm import tqdm
# write_root_1 = "C:\\LUNA\\Luna16_AugMask"
write_root_1 = "D:\\LUNA\\Luna16_AugSegData"
if not os.path.exists(write_root_1):
    # os.mkdir("C:\\LUNA\\Luna16_AugMask")
    os.mkdir("D:\\LUNA\\Luna16_AugSegData")
    
# read_root = "C:\\Users\\oplab\\Desktop\\Luna16_data\\Luna16_img"
read_root = "D:\\LUNA\\Luna16_AugData"
    
append = True

model, device = init_segment_model()

# split to train test
for sub_root in os.listdir(read_root):
    root = os.path.join(read_root, sub_root)
    if (glob.glob(os.path.join(write_root_1, sub_root)) != [] and sub_root != "subset0"):
        continue
    if not os.path.exists(os.path.join(write_root_1, sub_root)):
        os.mkdir(os.path.join(write_root_1, sub_root))
    print(root)
    for path in tqdm(os.listdir(root), desc="Augment"):
        if path.find("mhd") >= 0:
            raw_path_1 = os.path.join(write_root_1, sub_root, path)
            exist_path_1 =  glob.glob(os.path.join(write_root_1, sub_root, path))
            # print(exist_path_1)
            if (exist_path_1 != [] and append):
                # print("pass")
                continue
            # print(path)
            uid, _ = os.path.splitext(path)
            imagePath = os.path.join(root, path)
            ct_mhd = sitk.ReadImage(imagePath)
            if ct_mhd.GetDimension()==4 and ct_mhd.GetSize()[3]==1:
                ct_mhd = ct_mhd[...,0]
            origin = sitk.GetArrayFromImage(ct_mhd)
            segmented = segment(origin, model, device)
            hu_a = np.array(segmented, dtype=np.float32)
            # masks = masks_build(uid, origin)
            # augmented_mask = aug_mask(masks)
            # mask_a = np.array(augmented_mask, dtype=int)
            # Load or create a SimpleITK image
            current_origin = ct_mhd.GetOrigin()
            current_spacing = ct_mhd.GetSpacing()

            # sitk_image = sitk.GetImageFromArray(mask_a)
            sitk_image = sitk.GetImageFromArray(hu_a)
            # image_short = sitk.Cast(image, sitk.sitkInt16)

            # Set the image origin, spacing, and direction (modify as needed)
            sitk_image.SetOrigin((current_origin[0], current_origin[1], current_origin[2]))
            sitk_image.SetSpacing((current_spacing[0], current_spacing[1], current_spacing[2] / 2))

            # Save the image as a MetaImage file
            if not os.path.exists(os.path.join(write_root_1, sub_root)):
                os.mkdir(os.path.join(write_root_1, sub_root))
            # print(os.path.join(write_root_1, sub_root, path))
            sitk.WriteImage(sitk_image, raw_path_1)

D:\LUNA\Luna16_AugData\subset0


Augment: 100%|████████████████████████████████████████████████████████████████████| 178/178 [6:55:12<00:00, 139.96s/it]


D:\LUNA\Luna16_AugData\subset5


Augment: 100%|████████████████████████████████████████████████████████████████████| 178/178 [6:37:12<00:00, 133.89s/it]


D:\LUNA\Luna16_AugData\subset9


Augment: 100%|████████████████████████████████████████████████████████████████████| 176/176 [6:49:29<00:00, 139.60s/it]


In [2]:
uid = "1.3.6.1.4.1.14519.5.2.1.6279.6001.675543413149938600000570588203"
imagePath = "C://Users//oplab//Desktop//Luna16_data//Luna16_img//subset2//1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016233746780170740405.mhd"
ct_mhd = sitk.ReadImage(imagePath)
if ct_mhd.GetDimension()==4 and ct_mhd.GetSize()[3]==1:
    ct_mhd = ct_mhd[...,0]
origin = sitk.GetArrayFromImage(ct_mhd)
masks = masks_build(uid, origin)
augmented_mask = aug_mask(masks)

In [17]:
mask_a = np.array(augmented_mask, dtype=int)
# Load or create a SimpleITK image
current_origin = ct_mhd.GetOrigin()
current_spacing = ct_mhd.GetSpacing()

sitk_image = sitk.GetImageFromArray(mask_a)
# image_short = sitk.Cast(image, sitk.sitkInt16)

# Set the image origin, spacing, and direction (modify as needed)
sitk_image.SetOrigin((current_origin[0], current_origin[1], current_origin[2]))
sitk_image.SetSpacing((current_spacing[0], current_spacing[1], current_spacing[2] / 2))
raw_path_1 = "./mask/test.mhd"

sitk.WriteImage(sitk_image, raw_path_1) 

In [3]:
# uid = "test"
imagePath = "./mask/test.mhd"
ct_mhd = sitk.ReadImage(imagePath)
if ct_mhd.GetDimension()==4 and ct_mhd.GetSize()[3]==1:
    ct_mhd = ct_mhd[...,0]
mask = sitk.GetArrayFromImage(ct_mhd)
print(len(mask))
# masks = masks_build(uid, origin)
# augmented_mask = aug_mask(masks)

641


In [7]:
mask.astype(bool).sum()

73726

In [9]:
from PIL import Image, ImageDraw
def np2Png(np_arr, target_path):
    min_value = np.min(np_arr)
    max_value = np.max(np_arr)
    d = max_value - min_value if max_value - min_value > 0 else 1
    scaled_np_arr = (np_arr - min_value) / (d) * 255
    scaled_np_arr = scaled_np_arr.astype(np.uint8)
    slices = Image.fromarray(scaled_np_arr, mode='L')
    slices.save(target_path)

In [10]:
for i, mask_slice in enumerate(mask):
    np2Png(mask_slice.astype(bool).astype(int), "./mask/read_raw/{}.png".format(i))

In [15]:
for i, mask_slice in enumerate(augmented_mask):
    np2Png(mask_slice, "./mask/augmented/{}.png".format(i))
for i, mask_slice in enumerate(masks):
    np2Png(mask_slice.astype(float), "./mask/origin/{}.png".format(i))

In [12]:
os.path.join(write_root, sub_root, path)

NameError: name 'write_root' is not defined

In [ ]:
glob.glob(os.path.join(write_root, sub_root, path))

In [ ]:
sitk.WriteImage(sitk_image, "D://udet//augmented//{}.raw".format(series_uid)) 

In [ ]:
npy_path = glob.glob('./augmented_segmented/{}.npy'.format("1.3.6.1.4.1.14519.5.2.1.6279.6001.287966244644280690737019247886"))

In [ ]:
npy_path

In [ ]:
if (npy_path != [] and append):
        print("pass")

In [ ]:
np.save("./augmented_segmented/{}.npy".format(series_uid), hu_a)